In [6]:
a2 = [2, 2, 4, 3, 2, 0, 3, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 2,]
b2 = [3, 3, 4, 4, 2, 2, 3, 4, 3, 3, 4, 4, 4, 4, 3, 3, 3, 3,]
print(sum(a2), sum(b2))

50 59


In [7]:
a = [3, 2, 4, 3, 2, 4, 4, 3, 3, 3, 4, 4, 4, 3, 3, 3, 4, 2,]
b = [4, 3, 4, 4, 2, 2, 4, 4, 3, 3, 4, 4, 4, 4, 3, 3, 4, 3,]
print(sum(a), sum(b))

58 62


In [8]:
a3 = [4, 2, 4, 4, 2, 4, 4, 3, 3, 3, 4, 4, 4, 3, 3, 3, 4, 3,]
b3 = [3, 3, 4, 4, 2, 1, 4, 4, 3, 3, 4, 4, 4, 4, 3, 3, 4, 3,]
print(sum(a3), sum(b3))

61 60


In [24]:
a4 = [4, 4, 4, 4, 4, 4, 3, 4, 3, 3, 4, 4, 3, 4, 3, 3, 4, 4,]
b4 = [4, 4, 4, 4, 4, 4, 3, 4, 3, 3, 4, 4, 3, 4, 3, 3, 4, 4,]
print(sum(a4), sum(b4))

66 66


In [25]:
a5 = [4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 4, 4, 4, 4, 3, 3, 4, 4,]
b5 = [4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 4, 4, 4, 4, 3, 3, 4, 4,]
print(sum(a5), sum(b5))

68 68


In [13]:
import pandas as pd
import numpy as np
'''
Review the table labeled Figure 1: Training Dataset. Attribute 3 is continuous. Assume that we want to use a decision tree for modeling the data.

Using entropy as the measure of node impurity to calculate information gain, which attribute will provide the best split?
'''

# Create the data
data = {
    'Attribute1': ['T', 'T', 'T', 'F', 'F', 'F', 'F', 'T', 'F'],
    'Attribute2': ['T', 'T', 'F', 'F', 'T', 'T', 'F', 'F', 'T'],
    'Attribute3': [1, 6, 5, 4, 7, 3, 8, 7, 5],
    'Class': ['Y', 'Y', 'N', 'Y', 'N', 'N', 'N', 'Y', 'N']
}

# Convert to DataFrame
df = pd.DataFrame(data)

def entropy(y):
    class_labels = y.unique()
    entropy = 0
    for label in class_labels:
        p_label = len(y[y == label]) / len(y)
        entropy -= p_label * np.log2(p_label)
    return entropy

def information_gain(df, split_attribute_name, target_name="Class"):
    # Calculate the entropy of the whole dataset
    total_entropy = entropy(df[target_name])
    
    # Values and counts of the split attribute
    vals, counts = np.unique(df[split_attribute_name], return_counts=True)
    
    # Weighted entropy after split
    weighted_entropy = sum((counts[i] / sum(counts)) * 
                           entropy(df.where(df[split_attribute_name] == vals[i]).dropna()[target_name])
                           for i in range(len(vals)))
    
    # Information gain is the difference in entropy before and after the split
    info_gain = total_entropy - weighted_entropy
    return info_gain

gain_attribute1 = information_gain(df, 'Attribute1')
gain_attribute2 = information_gain(df, 'Attribute2')

# For Attribute3 (continuous), we need to handle it differently
# Here, we assume a binary split at a specific value, for simplicity, we'll use the median
median_value = df['Attribute3'].median()
df['Attribute3_split'] = df['Attribute3'] <= median_value
gain_attribute3 = information_gain(df, 'Attribute3_split')

# Print the information gains
print(f"Information Gain for Attribute 1: {gain_attribute1}")
print(f"Information Gain for Attribute 2: {gain_attribute2}")
print(f"Information Gain for Attribute 3 (split at median): {gain_attribute3}")

Information Gain for Attribute 1: 0.22943684069673975
Information Gain for Attribute 2: 0.007214618474517431
Information Gain for Attribute 3 (split at median): 0.007214618474517431


In [12]:
'''
Using entropy as the measure of node impurity and the information gain metric, what is the best split point for Attribute 3?
'''

# Function to calculate information gain for a given split point
def information_gain_continuous(df, split_attribute_name, split_value, target_name="Class"):
    # Calculate the entropy of the whole dataset
    total_entropy = entropy(df[target_name])
    
    # Split the dataset
    df_left = df[df[split_attribute_name] <= split_value]
    df_right = df[df[split_attribute_name] > split_value]
    
    # Calculate the entropy for each subset
    left_entropy = entropy(df_left[target_name])
    right_entropy = entropy(df_right[target_name])
    
    # Calculate the weighted average entropy after split
    weighted_entropy = (len(df_left) / len(df)) * left_entropy + (len(df_right) / len(df)) * right_entropy
    
    # Information gain is the difference in entropy before and after the split
    info_gain = total_entropy - weighted_entropy
    return info_gain
    
# Find the unique values in Attribute 3 and sort them
unique_values = sorted(df['Attribute3'].unique())

# Calculate the midpoints between consecutive unique values
split_points = [(unique_values[i] + unique_values[i+1]) / 2 for i in range(len(unique_values)-1)]

# Evaluate information gain for each split point
best_split = None
best_info_gain = -1

for split_point in split_points:
    info_gain = information_gain_continuous(df, 'Attribute3', split_point)
    print(f"Split at {split_point:.2f} gives information gain of {info_gain:.5f}")
    if info_gain > best_info_gain:
        best_info_gain = info_gain
        best_split = split_point

# Output the best split point
print(f"Best split point for Attribute 3 is at {best_split} with information gain of {best_info_gain}")

Split at 2.00 gives information gain of 0.14269
Split at 3.50 gives information gain of 0.00257
Split at 4.50 gives information gain of 0.07278
Split at 5.50 gives information gain of 0.00721
Split at 6.50 gives information gain of 0.01831
Split at 7.50 gives information gain of 0.10219
Best split point for Attribute 3 is at 2.0 with information gain of 0.14269027946047552


In [16]:
'''
Using the misclassification error as the measure of node impurity, which attribute, between Attribute 1 and Attribute 2, provides the best split? What is the misclassification error rate for the split?

The attribute with the lower misclassification error is the better attribute for splitting the data, as it results in fewer misclassified instances in the resulting nodes.
'''
data2 = {
    'Attribute1': ['T', 'T', 'T', 'F', 'F', 'F', 'F', 'T', 'F'],
    'Attribute2': ['T', 'T', 'F', 'F', 'T', 'T', 'F', 'F', 'T'],
    'Attribute3': [1, 6, 5, 4, 7, 3, 8, 7, 5],
    'Class': ['Y', 'Y', 'N', 'Y', 'N', 'N', 'N', 'Y', 'N']
}

df2 = pd.DataFrame(data2)

def misclassification_error(y):
    class_counts = y.value_counts()
    max_class = class_counts.max()
    error = 1 - max_class / len(y)
    return error

def misclassification_error_split(df, split_attribute_name, target_name="Class"):
    # Values of the split attribute
    vals = df[split_attribute_name].unique()
    
    # Calculate the weighted misclassification error
    weighted_error = 0
    for val in vals:
        subset = df[df[split_attribute_name] == val][target_name]
        subset_error = misclassification_error(subset)
        weighted_error += len(subset) / len(df) * subset_error
    
    return weighted_error

# Calculate misclassification error for Attribute 1 and Attribute 2
error_attribute1 = misclassification_error_split(df2, 'Attribute1')
error_attribute2 = misclassification_error_split(df2, 'Attribute2')

# Print the misclassification errors
print(f"Misclassification Error for Attribute 1: {error_attribute1}")
print(f"Misclassification Error for Attribute 2: {error_attribute2}")

Misclassification Error for Attribute 1: 0.2222222222222222
Misclassification Error for Attribute 2: 0.4444444444444444


In [18]:
'''
Figure 1: Training Dataset
Instance
Attribute 1
Attribute 2
Attribute 3
Class
1	T	T	1	Y
2	T	T	6	Y
3	T	F	5	N
4	F	F	4	Y
5	F	T	7	N
6	F	T	3	N
7	F	F	8	N
8	T	F	7	Y
9	F	T	5	N
 

Review the table labeled Figure 1: Training Dataset. Assume that we want to use a decision tree for modeling the data.

Using the Gini Index as the measure of node impurity, which attribute, between Attribute 1 and Attribute 2, provides the best split? What is the Gini Index for the split?

After running the code, you will get the Gini Index values for both attributes. The attribute with the lower Gini Index provides the best split.
'''
# Create the data
data3 = {
    'Attribute1': ['T', 'T', 'T', 'F', 'F', 'F', 'F', 'T', 'F'],
    'Attribute2': ['T', 'T', 'F', 'F', 'T', 'T', 'F', 'F', 'T'],
    'Attribute3': [1, 6, 5, 4, 7, 3, 8, 7, 5],
    'Class': ['Y', 'Y', 'N', 'Y', 'N', 'N', 'N', 'Y', 'N']
}

# Convert to DataFrame
df3 = pd.DataFrame(data3)

def gini_index(y):
    class_counts = y.value_counts()
    total_instances = len(y)
    gini = 1.0
    for count in class_counts:
        gini -= (count / total_instances) ** 2
    return gini

def gini_index_split(df, split_attribute_name, target_name="Class"):
    # Values of the split attribute
    vals = df[split_attribute_name].unique()
    
    # Calculate the weighted Gini index
    weighted_gini = 0
    for val in vals:
        subset = df[df[split_attribute_name] == val][target_name]
        subset_gini = gini_index(subset)
        weighted_gini += len(subset) / len(df) * subset_gini
    
    return weighted_gini

# Calculate Gini index for Attribute 1 and Attribute 2
gini_attribute1 = gini_index_split(df3, 'Attribute1')
gini_attribute2 = gini_index_split(df3, 'Attribute2')

# Print the Gini Index results
print(f"Gini Index for Attribute 1: {gini_attribute1}")
print(f"Gini Index for Attribute 2: {gini_attribute2}")

Gini Index for Attribute 1: 0.34444444444444433
Gini Index for Attribute 2: 0.4888888888888889


In [19]:
'''
Figure 2: Model Predictions for Images
Ground Truth
apple	apple	apple	apple	apple	apple	apple	apple	apple	apple
Predict
apple	pear	pear	apple	apple	pear	pear	pear	apple	apple
 

Review the table labeled Figure 2: Model Predictions for Images. You trained a model to predict types of fruits in images. The model is tested against 10 images which only include apples in them. The model predictions are recorded in the table.

What is the model precision and recall for ‘apple’ class?

Group of answer choices

Precision – 1 and recall – 1

Precision – 0.5 and recall – 0.5

Precision – 1 and recall – 0.5

Precision – 0.5 and recall – 1
'''
# Ground truth and predictions
ground_truth = ['apple', 'apple', 'apple', 'apple', 'apple', 'apple', 'apple', 'apple', 'apple', 'apple']
predictions = ['apple', 'pear', 'pear', 'apple', 'apple', 'pear', 'pear', 'pear', 'apple', 'apple']

# Calculate TP, FP, and FN
TP = sum((gt == 'apple') and (pred == 'apple') for gt, pred in zip(ground_truth, predictions))
FP = sum((gt != 'apple') and (pred == 'apple') for gt, pred in zip(ground_truth, predictions))  # In this case, FP is 0 because all ground truth are apples
FN = sum((gt == 'apple') and (pred != 'apple') for gt, pred in zip(ground_truth, predictions))

# Calculate Precision and Recall
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0

precision, recall


(1.0, 0.5)

In [22]:
'''
Figure 2: Model Predictions for Images
Ground Truth
apple	apple	apple	apple	apple	apple	apple	apple	apple	apple
Predict
apple	pear	pear	apple	apple	pear	pear	pear	apple	apple
 

Review the table labeled Figure 2: Model Predictions for Images. You trained a model to predict types of fruits in images. The model is tested against 10 images which only include apples in them. The model predictions are recorded in the table.

What is the model F1 score for the “apple” class?

Your answer should be in format X.XX or X.X (depending on the precision required on decimal places).
'''

# Ground truth and predictions
ground_truth = ['apple', 'apple', 'apple', 'apple', 'apple', 'apple', 'apple', 'apple', 'apple', 'apple']
predictions = ['apple', 'pear', 'pear', 'apple', 'apple', 'pear', 'pear', 'pear', 'apple', 'apple']

# Calculate TP, FP, and FN
TP = sum((gt == 'apple') and (pred == 'apple') for gt, pred in zip(ground_truth, predictions))
FP = sum((gt != 'apple') and (pred == 'apple') for gt, pred in zip(ground_truth, predictions))  # In this case, FP is 0
FN = sum((gt == 'apple') and (pred != 'apple') for gt, pred in zip(ground_truth, predictions))

# Calculate Precision and Recall
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0

# Calculate F1 Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Output the F1 Score formatted to two decimal places
f1_score_rounded = round(f1_score, 2)
f1_score, f1_score_rounded

(0.6666666666666666, 0.67)

In [23]:
'''
Consider the cost matrix shown above. Consider a machine M1 that out of 10,000 faulty parts classifies 7000 as faulty and 3000 as not faulty. Out of 10,000 not faulty parts classifies 4000 as faulty and 6000 as not faulty. Consider another machine M2, that out of the same 10,000 faulty parts classifies 8000 as faulty and 2000 as not faulty. Out of 10,000 not faulty parts classifies 5000 as faulty and 5000 as not faulty.

Which machine will you choose?

Predicted Faulty
Predicted Not Faulty
Faulty
0	10.0
Not Faulty

Understanding the Cost Matrix:
Faulty Part classified as Faulty: Cost = 0
Faulty Part classified as Not Faulty: Cost = 10.0
Not Faulty Part classified as Faulty: Cost = 1.0
Not Faulty Part classified as Not Faulty: Cost = 0

Machine M1 Performance:
Faulty parts: 7,000 correctly classified as Faulty, 3,000 incorrectly classified as Not Faulty.
Not Faulty parts: 4,000 incorrectly classified as Faulty, 6,000 correctly classified as Not Faulty.
'''

# Define the number of parts for each scenario
total_faulty_parts = 10000
total_not_faulty_parts = 10000

# Machine M1 performance
# M1 classifies 7000 faulty parts correctly, 3000 faulty parts incorrectly
M1_correct_faulty = 7000
M1_incorrect_faulty = 3000

# M1 classifies 6000 not faulty parts correctly, 4000 not faulty parts incorrectly
M1_correct_not_faulty = 6000
M1_incorrect_not_faulty = 4000

# Machine M2 performance
# M2 classifies 8000 faulty parts correctly, 2000 faulty parts incorrectly
M2_correct_faulty = 8000
M2_incorrect_faulty = 2000

# M2 classifies 5000 not faulty parts correctly, 5000 not faulty parts incorrectly
M2_correct_not_faulty = 5000
M2_incorrect_not_faulty = 5000

# Cost matrix
# Cost of classifying a faulty part as not faulty
cost_faulty_as_not_faulty = 10.0

# Cost of classifying a not faulty part as faulty
cost_not_faulty_as_faulty = 1.0

# Calculate the total cost for M1
cost_M1 = (M1_incorrect_faulty * cost_faulty_as_not_faulty) + (M1_incorrect_not_faulty * cost_not_faulty_as_faulty)

# Calculate the total cost for M2
cost_M2 = (M2_incorrect_faulty * cost_faulty_as_not_faulty) + (M2_incorrect_not_faulty * cost_not_faulty_as_faulty)

# Print out the total costs for both machines
print(f"Total Cost for M1: {cost_M1}")
print(f"Total Cost for M2: {cost_M2}")

# Determine which machine has the lower cost
if cost_M1 < cost_M2:
    print("Choose M1")
elif cost_M2 < cost_M1:
    print("Choose M2")
else:
    print("Either machine can be chosen")


Total Cost for M1: 34000.0
Total Cost for M2: 25000.0
Choose M2


In [1]:
import pandas as pd
import numpy as np

# Define the dataset
data = {
    'Attribute 1': ['T', 'T', 'T', 'F', 'F', 'F', 'F', 'T', 'F'],
    'Attribute 2': ['T', 'T', 'F', 'F', 'T', 'T', 'F', 'F', 'T'],
    'Attribute 3': [1, 6, 5, 4, 7, 3, 8, 7, 5],
    'Class': ['Y', 'Y', 'N', 'Y', 'N', 'N', 'N', 'Y', 'N']
}

# Create DataFrame
df = pd.DataFrame(data)

# Function to calculate entropy
def entropy(probs):
    return -sum([p * np.log2(p) for p in probs if p > 0])

# Function to calculate entropy of the dataset
def entropy_of_dataset(df, target_column):
    target_values = df[target_column].value_counts(normalize=True)
    return entropy(target_values)

# Function to calculate information gain
def information_gain(df, split_attribute, target_attribute='Class'):
    total_entropy = entropy_of_dataset(df, target_attribute)
    
    # Get the unique values of the attribute we are splitting by
    values = df[split_attribute].unique()
    
    # Calculate the weighted entropy after the split
    weighted_entropy = 0
    for value in values:
        subset = df[df[split_attribute] == value]
        subset_entropy = entropy_of_dataset(subset, target_attribute)
        weighted_entropy += (len(subset) / len(df)) * subset_entropy
    
    # Information gain is the reduction in entropy
    return total_entropy - weighted_entropy

# Entropy before any split
initial_entropy = entropy_of_dataset(df, 'Class')
print(f"Initial Entropy: {initial_entropy}")

# Calculate information gain for Attribute 1 and Attribute 2
gain_attribute_1 = information_gain(df, 'Attribute 1')
gain_attribute_2 = information_gain(df, 'Attribute 2')

print(f"Information Gain for Attribute 1: {gain_attribute_1}")
print(f"Information Gain for Attribute 2: {gain_attribute_2}")

# To handle Attribute 3 (continuous), we need to find the best threshold for splitting.
# Let's calculate the gain for each possible threshold:
def information_gain_continuous(df, attribute, target_attribute='Class'):
    # Sort by the continuous attribute
    df_sorted = df.sort_values(attribute)
    values = df_sorted[attribute].unique()
    
    best_gain = -1
    best_threshold = None
    
    # Try every possible split point (midpoints between adjacent values)
    for i in range(len(values) - 1):
        threshold = (values[i] + values[i + 1]) / 2
        
        # Create two subsets: less than and greater than or equal to the threshold
        df_left = df[df[attribute] <= threshold]
        df_right = df[df[attribute] > threshold]
        
        # Calculate weighted entropy
        left_entropy = entropy_of_dataset(df_left, target_attribute)
        right_entropy = entropy_of_dataset(df_right, target_attribute)
        weighted_entropy = (len(df_left) / len(df)) * left_entropy + (len(df_right) / len(df)) * right_entropy
        
        # Information gain for this split
        gain = initial_entropy - weighted_entropy
        
        if gain > best_gain:
            best_gain = gain
            best_threshold = threshold
    
    return best_gain, best_threshold

gain_attribute_3, threshold_3 = information_gain_continuous(df, 'Attribute 3')
print(f"Information Gain for Attribute 3: {gain_attribute_3} (Threshold: {threshold_3})")


Initial Entropy: 0.9910760598382222
Information Gain for Attribute 1: 0.22943684069673975
Information Gain for Attribute 2: 0.007214618474517431
Information Gain for Attribute 3: 0.14269027946047552 (Threshold: 2.0)
